In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import warnings

# Suppress specific sklearn warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
with open('face_model_gb.pkl', 'rb') as f:
    face_model = pickle.load(f)
with open('pose_model_rf.pkl', 'rb') as f:
    pose_model = pickle.load(f)
    
print(face_model)
print(pose_model)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])


In [4]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
#             # Concate rows
#             row = pose_row+face_row
            
            # Make face Detections
            face_X = pd.DataFrame([face_row])
            face_class = face_model.predict(face_X)[0]
            face_prob = face_model.predict_proba(face_X)[0]
            print(face_class, face_prob)

            # Make pose Detections
            pose_X = pd.DataFrame([pose_row])
            pose_class = pose_model.predict(pose_X)[0]
            pose_prob = pose_model.predict_proba(pose_X)[0]
            print(pose_class, pose_prob)
             
            # Grab left ear coords for pose class
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(pose_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, pose_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Grab right ear coords for pose class
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(face_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, face_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
#             # Get status box
#             cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
#             # Display Class
#             cv2.putText(image, 'CLASS'
#                         , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, pose_class.split(' ')[0]
#           , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
#             # Display Probability
#             cv2.putText(image, 'PROB'
#                         , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, str(round(pose_prob[np.argmax(pose_prob)],2))
#                         , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

frown [0.97667352 0.02332648]
hands pocket [0.4 0.6]
frown [0.97667352 0.02332648]
hands pocket [0.4 0.6]
frown [0.97409755 0.02590245]
hands pocket [0.4 0.6]
frown [0.96081631 0.03918369]
hands pocket [0.4 0.6]
frown [0.96081631 0.03918369]
hands pocket [0.39 0.61]
frown [0.97301099 0.02698901]
hands pocket [0.39 0.61]
frown [0.96081631 0.03918369]
hands pocket [0.38 0.62]
frown [0.96081631 0.03918369]
hands pocket [0.38 0.62]
frown [0.96081631 0.03918369]
hands pocket [0.38 0.62]
frown [0.96081631 0.03918369]
hands pocket [0.37 0.63]
frown [0.96487294 0.03512706]
hands pocket [0.37 0.63]
frown [0.96487294 0.03512706]
hands pocket [0.25 0.75]
frown [0.96487294 0.03512706]
hands pocket [0.3 0.7]
frown [0.96240656 0.03759344]
hands pocket [0.39 0.61]
frown [0.95306742 0.04693258]
arms crossed [0.91 0.09]
frown [0.9519897 0.0480103]
arms crossed [0.92 0.08]
frown [0.94028524 0.05971476]
arms crossed [0.93 0.07]
frown [0.9340854 0.0659146]
arms crossed [0.97 0.03]
frown [0.95031793 0.0496

frown [0.9502274 0.0497726]
hands pocket [0.48 0.52]
frown [0.96644529 0.03355471]
hands pocket [0.38 0.62]
frown [0.96644529 0.03355471]
hands pocket [0.07 0.93]
frown [0.92011419 0.07988581]
hands pocket [0.01 0.99]
frown [0.95335957 0.04664043]
arms crossed [0.56 0.44]
frown [0.97229683 0.02770317]
arms crossed [0.54 0.46]
frown [0.96721468 0.03278532]
arms crossed [0.55 0.45]
frown [0.92771177 0.07228823]
arms crossed [0.55 0.45]
frown [0.94441806 0.05558194]
arms crossed [0.58 0.42]
frown [0.97611239 0.02388761]
arms crossed [0.61 0.39]
frown [0.96571438 0.03428562]
arms crossed [0.68 0.32]
frown [0.94638199 0.05361801]
arms crossed [0.7 0.3]
frown [0.93913362 0.06086638]
arms crossed [0.72 0.28]
frown [0.96183446 0.03816554]
arms crossed [0.74 0.26]
frown [0.96529215 0.03470785]
arms crossed [0.74 0.26]
frown [0.9561077 0.0438923]
arms crossed [0.76 0.24]
frown [0.96183446 0.03816554]
arms crossed [0.78 0.22]
frown [0.94663443 0.05336557]
arms crossed [0.81 0.19]
frown [0.9466344

frown [0.94591156 0.05408844]
hands pocket [0.43 0.57]
frown [0.97377265 0.02622735]
hands pocket [0.43 0.57]
frown [0.96502565 0.03497435]
hands pocket [0.41 0.59]
frown [0.9782615 0.0217385]
hands pocket [0.4 0.6]
frown [0.96399013 0.03600987]
hands pocket [0.37 0.63]
frown [0.97217668 0.02782332]
hands pocket [0.38 0.62]
frown [0.97084392 0.02915608]
hands pocket [0.39 0.61]
frown [0.94349077 0.05650923]
hands pocket [0.35 0.65]
frown [0.94293016 0.05706984]
hands pocket [0.3 0.7]
frown [0.95716849 0.04283151]
hands pocket [0.3 0.7]
frown [0.95358933 0.04641067]
hands pocket [0.2 0.8]
frown [0.96459241 0.03540759]
hands pocket [0.03 0.97]
frown [0.96281764 0.03718236]
hands pocket [0.02 0.98]
frown [0.9765037 0.0234963]
hands pocket [0.03 0.97]
frown [0.97246077 0.02753923]
hands pocket [0.04 0.96]
frown [0.9724269 0.0275731]
hands pocket [0.04 0.96]
frown [0.95748449 0.04251551]
hands pocket [0.03 0.97]
frown [0.96296902 0.03703098]
hands pocket [0.01 0.99]
frown [0.97407702 0.0259

frown [0.98496047 0.01503953]
hands pocket [0.37 0.63]
frown [0.98226701 0.01773299]
hands pocket [0.37 0.63]
frown [0.97968025 0.02031975]
hands pocket [0.37 0.63]
frown [0.93423645 0.06576355]
hands pocket [0.37 0.63]
frown [0.93566786 0.06433214]
hands pocket [0.38 0.62]
frown [0.75256976 0.24743024]
hands pocket [0.37 0.63]
frown [0.85767234 0.14232766]
hands pocket [0.37 0.63]
frown [0.8672155 0.1327845]
hands pocket [0.37 0.63]
frown [0.84500271 0.15499729]
hands pocket [0.37 0.63]
frown [0.86989808 0.13010192]
hands pocket [0.37 0.63]
frown [0.87175099 0.12824901]
hands pocket [0.37 0.63]
frown [0.86158661 0.13841339]
hands pocket [0.37 0.63]
frown [0.87578022 0.12421978]
hands pocket [0.37 0.63]
frown [0.84777242 0.15222758]
hands pocket [0.37 0.63]
frown [0.88326865 0.11673135]
hands pocket [0.37 0.63]
frown [0.88956493 0.11043507]
hands pocket [0.37 0.63]
frown [0.86185077 0.13814923]
hands pocket [0.37 0.63]
frown [0.92207468 0.07792532]
hands pocket [0.37 0.63]
frown [0.876

frown [0.96730507 0.03269493]
hands pocket [0.05 0.95]
frown [0.89486655 0.10513345]
hands pocket [0.05 0.95]
frown [0.98392691 0.01607309]
hands pocket [0.05 0.95]
frown [0.98936152 0.01063848]
hands pocket [0.35 0.65]
frown [0.98966782 0.01033218]
hands pocket [0.37 0.63]
frown [0.9803986 0.0196014]
hands pocket [0.37 0.63]
frown [0.99535884 0.00464116]
hands pocket [0.37 0.63]
frown [0.87912334 0.12087666]
hands pocket [0.38 0.62]
frown [0.87629354 0.12370646]
hands pocket [0.38 0.62]
frown [0.92640643 0.07359357]
hands pocket [0.38 0.62]
frown [0.92640643 0.07359357]
hands pocket [0.24 0.76]
frown [0.92640643 0.07359357]
hands pocket [0.19 0.81]
frown [0.91828374 0.08171626]
hands pocket [0.18 0.82]
frown [0.90618219 0.09381781]
hands pocket [0.23 0.77]
frown [0.90970956 0.09029044]
hands pocket [0.23 0.77]
frown [0.89932352 0.10067648]
hands pocket [0.27 0.73]
frown [0.86433623 0.13566377]
hands pocket [0.31 0.69]
frown [0.88153161 0.11846839]
hands pocket [0.32 0.68]
frown [0.904

...

smile [0.11595768 0.88404232]
hands pocket [0.4 0.6]
frown [0.66244035 0.33755965]
hands pocket [0.4 0.6]
smile [0.0224599 0.9775401]
hands pocket [0.4 0.6]
smile [0.4349404 0.5650596]
hands pocket [0.4 0.6]
smile [0.29342358 0.70657642]
hands pocket [0.4 0.6]
frown [0.88842215 0.11157785]
hands pocket [0.4 0.6]
smile [0.46424177 0.53575823]
hands pocket [0.4 0.6]
frown [0.60443844 0.39556156]
hands pocket [0.4 0.6]
frown [0.55675839 0.44324161]
hands pocket [0.4 0.6]
frown [0.65227042 0.34772958]
hands pocket [0.4 0.6]
frown [0.52233409 0.47766591]
hands pocket [0.4 0.6]
frown [0.72802241 0.27197759]
hands pocket [0.4 0.6]
smile [0.23452557 0.76547443]
hands pocket [0.4 0.6]
frown [0.76260368 0.23739632]
hands pocket [0.4 0.6]
frown [0.95565465 0.04434535]
hands pocket [0.4 0.6]
frown [0.97319491 0.02680509]
hands pocket [0.4 0.6]
frown [0.96904485 0.03095515]
hands pocket [0.4 0.6]
frown [0.97054387 0.02945613]
hands pocket [0.4 0.6]
frown [0.94420499 0.05579501]
hands pocket [0.4 0.

frown [0.64703031 0.35296969]
hands pocket [0.4 0.6]
frown [0.86721702 0.13278298]
hands pocket [0.44 0.56]
frown [0.91154118 0.08845882]
hands pocket [0.45 0.55]
frown [0.87628076 0.12371924]
hands pocket [0.42 0.58]
frown [0.81625123 0.18374877]
hands pocket [0.42 0.58]
frown [0.64592749 0.35407251]
hands pocket [0.41 0.59]
frown [0.65527161 0.34472839]
hands pocket [0.4 0.6]
frown [0.5926815 0.4073185]
hands pocket [0.42 0.58]
frown [0.64324565 0.35675435]
hands pocket [0.43 0.57]
frown [0.91050119 0.08949881]
hands pocket [0.43 0.57]
frown [0.90656866 0.09343134]
hands pocket [0.42 0.58]
frown [0.68813203 0.31186797]
hands pocket [0.4 0.6]
frown [0.91099443 0.08900557]
hands pocket [0.4 0.6]
frown [0.79258826 0.20741174]
hands pocket [0.44 0.56]
frown [0.78863486 0.21136514]
arms crossed [0.51 0.49]
frown [0.81240705 0.18759295]
arms crossed [0.53 0.47]
frown [0.82308908 0.17691092]
arms crossed [0.52 0.48]
frown [0.79258826 0.20741174]
arms crossed [0.52 0.48]
frown [0.79258826 0.

smile [0.0268472 0.9731528]
hands pocket [0.37 0.63]
smile [0.0210209 0.9789791]
hands pocket [0.37 0.63]
smile [0.0365112 0.9634888]
hands pocket [0.37 0.63]
smile [0.03070999 0.96929001]
hands pocket [0.37 0.63]
smile [0.02611785 0.97388215]
hands pocket [0.37 0.63]
smile [0.04206099 0.95793901]
hands pocket [0.37 0.63]
smile [0.03528744 0.96471256]
hands pocket [0.37 0.63]
smile [0.03953821 0.96046179]
hands pocket [0.37 0.63]
smile [0.01219197 0.98780803]
hands pocket [0.37 0.63]
smile [0.03677364 0.96322636]
hands pocket [0.39 0.61]
frown [0.84521433 0.15478567]
hands pocket [0.4 0.6]
frown [0.93704998 0.06295002]
hands pocket [0.4 0.6]
frown [0.95656796 0.04343204]
hands pocket [0.39 0.61]
frown [0.95656796 0.04343204]
hands pocket [0.4 0.6]
frown [0.94934768 0.05065232]
hands pocket [0.4 0.6]
smile [0.11800516 0.88199484]
hands pocket [0.39 0.61]
smile [0.10699268 0.89300732]
hands pocket [0.39 0.61]
smile [0.1446314 0.8553686]
hands pocket [0.39 0.61]
smile [0.13723977 0.862760

smile [0.03597704 0.96402296]
hands pocket [0.37 0.63]
smile [0.04567943 0.95432057]
hands pocket [0.37 0.63]
smile [0.04114998 0.95885002]
hands pocket [0.37 0.63]
smile [0.11973448 0.88026552]
hands pocket [0.37 0.63]
smile [0.09677618 0.90322382]
hands pocket [0.37 0.63]
smile [0.09677618 0.90322382]
hands pocket [0.37 0.63]
smile [0.11973448 0.88026552]
hands pocket [0.37 0.63]
smile [0.12236404 0.87763596]
hands pocket [0.37 0.63]
smile [0.08434317 0.91565683]
hands pocket [0.37 0.63]
smile [0.21894637 0.78105363]
hands pocket [0.37 0.63]
smile [0.2303829 0.7696171]
hands pocket [0.37 0.63]
smile [0.07701272 0.92298728]
hands pocket [0.37 0.63]
smile [0.09670606 0.90329394]
hands pocket [0.38 0.62]
smile [0.08153409 0.91846591]
hands pocket [0.37 0.63]
smile [0.09640877 0.90359123]
hands pocket [0.38 0.62]
smile [0.10929584 0.89070416]
hands pocket [0.38 0.62]
smile [0.13372465 0.86627535]
hands pocket [0.37 0.63]
smile [0.22683722 0.77316278]
hands pocket [0.37 0.63]
smile [0.213

KeyboardInterrupt: 